In [1]:
import torch
from tqdm import tqdm
from network.conv_node import NODE
from misc import *
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from torchmetrics.multimodal import CLIPImageQualityAssessment
import time

# GPU 번호 지정
gpu_number = 3  # 원하는 GPU 번호로 변경 가능

# GPU 사용 제한
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_number)

# 이제 GPU가 한 개만 보이므로 cuda:0으로 접근
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/home/lbw/miniconda3/envs/CLODE_128_bw/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model and Dataset (LOL train 485 images)

In [2]:

model = NODE(device, (3, 256, 256), 32, augment_dim=0, time_dependent=True, adjoint=True)
model.eval()
model.to(device)
model.load_state_dict(torch.load(f'/home/lbw/CLODE/pth/universal.pth', weights_only=True), strict=False)

file_path = Path('/home/lbw/data/our485')
# img_labels = sorted(os.listdir(file_path / 'low'))
img_labels = [f for f in sorted(os.listdir(file_path / 'low')) if f.lower().endswith('.png')]

def load_image(idx):
    lq_img = image_tensor(file_path / 'low' / img_labels[idx], size=(256, 256))
    gt_img = image_tensor(file_path / 'high' / img_labels[idx], size=(256, 256))
    
    return lq_img.to(device), gt_img.to(device)


In [3]:

prompts = ('brightness', 'noisiness', 'quality')  # 단순 튜플로 변경

clip_metric = CLIPImageQualityAssessment(
    model_name_or_path="openai/clip-vit-base-patch16",
    prompts=prompts  # 이미 적절한 형식을 가진 튜플
).to(device)

def calculate_clip_score(pred, prompts=prompts):    
    # 이미 배치 차원이 있는지 확인하고 없으면 추가
    if len(pred.shape) == 3:
        pred = pred.unsqueeze(0)
    
    with torch.no_grad():
        # 한 번의 forward pass로 모든 프롬프트에 대한 점수를 계산
        scores = clip_metric(pred)

    # 결과 반환 (scores는 리스트 형태로 반환됨)
    return scores[prompts[0]].item(), scores[prompts[1]].item(), scores[prompts[2]].item()

# 테스트
start = time.time()
a,b,c = calculate_clip_score(torch.rand(3, 256, 256).to(device))
print(f"Time: {time.time() - start}")
print(f"Brightness: {a}, Noisiness: {b}, Quality: {c}")

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Time: 0.22564435005187988
Brightness: 0.19360555708408356, Noisiness: 0.3640592396259308, Quality: 0.5380584001541138


Try find best T values by PSNR

In [ ]:

# 메인 루프 최적화
T_values = np.linspace(2, 5, 30)

results = []
brightness_scores = []
noisiness_scores = []
quality_scores = []

# T 값들을 먼저 텐서로 변환하여 반복 변환 방지
T_tensors = [torch.tensor([0, T]).float().cuda() for T in T_values]

with torch.no_grad():
    for idx in tqdm(range(len(img_labels))):
        lq_img, gt_img = load_image(idx)
        high_psnr = 0.0
        best_T = 2.0
        
        # 모든 T에 대한 예측을 한 번에 계산
        preds = []
        psnrs = []
        
        for i, T_tensor in enumerate(T_tensors):
            pred = model(lq_img, T_tensor, inference=True)['output'][0]
            preds.append(pred)
            psnr = calculate_psnr(pred, gt_img).item()
            psnrs.append(psnr)
            
            if high_psnr < psnr:
                high_psnr = psnr
                best_T = T_values[i]
        
        # 모든 예측에 대해 CLIP 점수 계산
        for i, pred in enumerate(preds):
            bright_score, noise_score, quality_score = calculate_clip_score(pred)
            brightness_scores.append([idx, T_values[i], bright_score])
            noisiness_scores.append([idx, T_values[i], noise_score])
            quality_scores.append([idx, T_values[i], quality_score])
        
        results.append([best_T, high_psnr])

save_path = Path('/home/lbw/CLODE/scores_csv')
save_path.mkdir(parents=True, exist_ok=True)

results = np.array(results)
np.save(Path(save_path / 'results.npy'), results)

brightness_scores = np.array(brightness_scores)
noisiness_scores = np.array(noisiness_scores)
quality_scores = np.array(quality_scores)

np.save(Path(save_path / 'brightness_scores.npy'), brightness_scores)
np.save(Path(save_path / 'noisiness_scores.npy'), noisiness_scores)
np.save(Path(save_path / 'quality_scores.npy'), quality_scores)



  1%|          | 5/485 [02:54<4:29:34, 33.70s/it]

Adapt $\alpha$, $\beta$, $\gamma$ so that weighted IQA score approximates score with optimal T

In [ ]:
prompts = ['brightness', 'noisiness', 'quality']
weights = [1.0, 1.0, 1.0]
clip_iqa = CLIPImageQualityAssessment(prompts=prompts).to(device)

learning_rate = 0.1
asc, desc = learning_rate * len(prompts) / (len(prompts) -1), learning_rate / (len(prompts) - 1)

def adjust_clip_weights(pred, weights):
    score = clip_iqa(pred.unsqueeze(0))
    scores = [score[prompt].item() for prompt in prompts]
    max_idx = np.argmax(scores)
    
    weights[max_idx] += asc
    weights -= desc
    
    return weights

with torch.no_grad():
    for idx in tqdm(range(len(img_labels))):
        lq_img, gt_img = load_image(idx)

        T = results[idx][0]
        integration_time = torch.tensor([0, T]).float().cuda()
        pred = model(lq_img, integration_time, inference=True)['output'][0]
            
        weights = adjust_clip_weights(pred, weights)
